<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

# 将字符串转为set
vocab = sorted(set(text))

print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])


[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
def split_input_target(id_text):
    # abcde -> abcd, bcde
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, 
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
    
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))


tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [0]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)

In [11]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.SimpleRNN(units = rnn_units,
                               return_sequences = True),
        keras.layers.Dense(vocab_size)
    ])
    
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units, 
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [13]:
# random sampling. 
# greedy策略; random策略.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[ 9]
 [57]
 [58]
 [32]
 [60]
 [52]
 [52]
 [58]
 [56]
 [30]
 [ 2]
 [48]
 [60]
 [14]
 [24]
 [ 1]
 [ 1]
 [47]
 [25]
 [32]
 [43]
 [45]
 [ 6]
 [64]
 [26]
 [ 4]
 [ 0]
 [14]
 [62]
 [14]
 [62]
 [53]
 [27]
 [33]
 [61]
 [61]
 [37]
 [21]
 [58]
 [51]
 [30]
 [16]
 [ 4]
 [41]
 [54]
 [45]
 [47]
 [56]
 [25]
 [24]
 [43]
 [ 0]
 [ 8]
 [33]
 [ 0]
 [55]
 [22]
 [64]
 [63]
 [15]
 [44]
 [16]
 [37]
 [ 1]
 [24]
 [31]
 [56]
 [ 2]
 [27]
 [33]
 [ 4]
 [39]
 [32]
 [15]
 [12]
 [63]
 [ 1]
 [52]
 [63]
 [53]
 [14]
 [34]
 [ 6]
 [16]
 [28]
 [12]
 [49]
 [64]
 [61]
 [49]
 [48]
 [32]
 [27]
 [25]
 [39]
 [55]
 [59]
 [57]
 [52]
 [57]], shape=(100, 1), dtype=int64)
tf.Tensor(
[ 9 57 58 32 60 52 52 58 56 30  2 48 60 14 24  1  1 47 25 32 43 45  6 64
 26  4  0 14 62 14 62 53 27 33 61 61 37 21 58 51 30 16  4 41 54 45 47 56
 25 24 43  0  8 33  0 55 22 64 63 15 44 16 37  1 24 31 56  2 27 33  4 39
 32 15 12 63  1 52 63 53 14 34  6 16 28 12 49 64 61 49 48 32 27 25 39 55
 59 57 52 57], shape=(100,), dtype=int64)


In [14]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  'ord: what is thy will with her?\n\nSLY:\nAre you my wife and will not call me husband?\nMy men should ca'

Output:  'rd: what is thy will with her?\n\nSLY:\nAre you my wife and will not call me husband?\nMy men should cal'

Predictions:  '3stTvnntrR!jvBL  iMTeg,zN&\nBxBxoOUwwYItmRD&cpgirMLe\n.U\nqJzyCfDY LSr!OU&aTC?y nyoBV,DP?kzwkjTOMaqusns'


In [0]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

In [16]:
model.compile(optimizer = 'adam', loss = loss)

example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1808476


In [17]:
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 100

history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 54s 313ms/step - loss: 2.7390
Epoch 2/100
172/172 [==============================] - 53s 308ms/step - loss: 2.0767
Epoch 3/100
172/172 [==============================] - 54s 312ms/step - loss: 1.8480
Epoch 4/100
172/172 [==============================] - 53s 311ms/step - loss: 1.7030
Epoch 5/100
172/172 [==============================] - 54s 312ms/step - loss: 1.6073
Epoch 6/100
172/172 [==============================] - 54s 314ms/step - loss: 1.5412
Epoch 7/100
172/172 [==============================] - 53s 310ms/step - loss: 1.4928
Epoch 8/100
172/172 [==============================] - 54s 311ms/step - loss: 1.4555
Epoch 9/100
172/172 [==============================] - 53s 310ms/step - loss: 1.4271
Epoch 10/100
172/172 [==============================] - 53s 311ms/step - loss: 1.4028
Epoch 11/100
172/172 [==============================] - 53s 311ms/step - loss: 1.3816
Epoch 12/100
172/172 [==============================] - 54s 311

In [18]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints/ckpt_100'

In [19]:
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model ->c
# B.append(c) -> C
# C(Abc) -> model -> ..
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [22]:
def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # predictions: [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions: [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predictions: [input_eval_len, 1]
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All")
        # 2. sample -> ch ->text_generated.
        # 3. undate input_eval
print(new_text)

All tlver
I sth de at bundime ds hat thake d haf he hie, OMEOME me che! n mmouthitralf ay, tlaton, veearofy, l pamupre chenghyou ier astheacont dobe. atlea be thapaliee wheat'da ix of NESTher y, th acow te'
'd:

IOMNUCie t s int'! Ison tan d bowiname;
ASIsell.
I no CA:


's ceromors coonoute s atyoze n kighthnge horamixell litoulegathof t r ntitheritt? lap d t ufupprg mor thmmashas pionth! dof arr ars hamme the t p lldrentarow s withaby ag thanourathee s t?
I in IEro therers, ERWhenoouthithito t sh owh Whofa breshil tho llup IINothioullthit!
Ty's:
IRCHoukith nul OFit
BERI ralon:

HOf: wine igborye'd andup sarasifure ICUCA, iote ser: h.
Tho as tothe ther d, CUF:
Mafel o d f ica t' w. me y nd, t me tas tharaleran;
STHAs th byore at ind tht

K:
ULIERDUCoug'd wind; t CUS:
ICo qur Se; STitie, t bitin:
IVEORETod t boul tap sthe Hye es vech the is icrbinnd.
AME
Whemet?
A:
RENove pouche ICEresishe lonjulf?
E othe ochy d tl t t IULo p, mastunak! fr tith n p pes ALIOROfrerd am n, toberth hel d'
